In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random

In [2]:
device='cuda'if torch.cuda.is_available()else'cpu'

In [3]:
torch.manual_seed(1)
random.seed(1)
if device=="cuda":
    torch.cuda.manual_seed_all(1)

In [4]:
learning_rate=0.003
batch_size=100
training_epochs=10

In [5]:
mnist_train=dsets.MNIST(root='Data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='Data/',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)

In [6]:
dataloader=torch.utils.data.DataLoader(dataset=mnist_train,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True)

Dropout 是一种常用的正则化技术，主要用于防止神经网络在训练过程中出现过拟合的问题。

它的核心思想是在每个训练批次中随机“丢弃”一部分神经元（即将它们的激活值设为0），使得网络不会过分依赖某些局部特征

    1.增强泛化能力： 每次训练时随机屏蔽部分神经元，让模型在不同的子网络上学习，从而减少对单个神经元的过度依赖。
    
    2.减少过拟合： 特别是在训练数据较少的情况下，Dropout 可以有效地抑制模型在训练集上的过拟合现象，从而在验证或测试集上获得更好的表现。

隐藏层宽度（神经元个数）

参数选择的建议：

1.模型容量与任务复杂性： 较大的隐藏层宽度可以让模型具有更强的表达能力，适合处理复杂任务。但如果数据量不够大或者任务本身较简单，过宽的隐藏层可能导致过拟合。

2.实验调优： 建议通过交叉验证、网格搜索或其他超参数调优方法，根据任务表现来选择最合适的神经元数量。

隐藏层深度（层数）：

层数越多越好吗？

优点： 更深的网络可以学习到更高层次、更抽象的特征，对于复杂任务来说有优势。

缺点： 层数增加也会带来梯度消失或梯度爆炸的问题，使得训练过程更为困难；同时，过深的网络更容易过拟合，并且训练时间和计算资源的消耗也会大幅增加。

设计建议：

1.适度为宜： 选择合适的深度应结合具体任务、数据量和计算资源。实践中，深度神经网络的设计往往需要反复试验。

2.辅助技术： 如果想构建更深的网络，可以考虑使用一些现代技术，如残差连接（ResNet）、Batch Normalization 等，这些方法可以缓解深层网络训练中常见的问题。

In [7]:
#linear1=torch.nn.Linear(784,256,bias=True).to(device)
#linear2=torch.nn.Linear(256,256,bias=True).to(device)
#linear3=torch.nn.Linear(256,10,bias=True).to(device)

# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
selu=torch.nn.SELU()
dropout=torch.nn.Dropout(0.1)#p=drop_prob

为什么在 forward 中需要激活函数？

1.引入非线性：
如果仅仅使用线性变换，无论叠加多少层，整个网络仍然等价于一个线性模型，无法拟合复杂的非线性关系。激活函数的引入使得模型可以表达任意复杂的映射关系，从而能够解决更复杂的问题。

2.增强模型表达能力：
激活函数能够让网络层与层之间建立非线性映射，这对于模式识别和函数逼近都是必不可少的。

In [8]:
#model=torch.nn.Sequential(linear1,relu,linear2,relu,linear3).to(device)

model = torch.nn.Sequential(linear1, selu, dropout,
                            linear2, selu, dropout,
                            linear3, selu, dropout,
                            linear4, selu, dropout,
                            linear5).to(device)

在神经网络中，权重（weights）决定了各个输入特征如何组合并影响输出，而偏置（bias）起到平移输出的作用。

在深度神经网络中，权重的初始值对训练效果起着至关重要的作用。如果初始化不当，可能会导致：

   1.梯度消失或爆炸：多层网络中，梯度可能随着层数的增加不断变小或变大，从而导致网络难以训练。

   2.网络收敛慢或陷入局部最优。

使用 init_normal 随机初始化权重主要是为了打破对称性，使得每个神经元一开始就处于不同的状态，从而能学习到不同的特征。
随机初始化可以帮助网络从不同角度开始优化，避免多个神经元更新方向完全一致。

偏置通常初始化为0或一个很小的常数值，因为它们只负责调整激活函数的输出，不会引起神经元间的对称问题。简单的初始化（例如全0）通常已经足够。

In [9]:
#torch.nn.init.kaiming_normal_(linear1.weight,nonlinearity='relu')
#torch.nn.init.kaiming_normal_(linear2.weight,nonlinearity='relu')
#torch.nn.init.kaiming_normal_(linear3.weight,nonlinearity='relu')

#torch.nn.init.xavier_uniform_(linear1.weight)
#torch.nn.init.xavier_uniform_(linear2.weight)
#torch.nn.init.xavier_uniform_(linear3.weight)

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0147, -0.0003, -0.0210,  ...,  0.0707, -0.0314, -0.0136],
        [ 0.0718, -0.0103,  0.0366,  ..., -0.0319,  0.0462,  0.0303],
        [ 0.0575, -0.0890, -0.0492,  ...,  0.0100,  0.0807, -0.0359],
        ...,
        [-0.0403,  0.0531, -0.0981,  ...,  0.0617, -0.0011,  0.0624],
        [-0.0896,  0.0671,  0.0815,  ...,  0.0659,  0.1023, -0.0633],
        [ 0.1020,  0.0481, -0.0295,  ..., -0.1030, -0.0380, -0.0858]],
       requires_grad=True)

In [10]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [11]:
total_batch=len(dataloader)#the num of mini_batch
print(total_batch)
model.train()
for epoch in range(training_epochs+1):
    avg_cost=0
    for X,Y in dataloader:
        X=X.view(-1,28*28).to(device)
        Y=Y.to(device)
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost+=cost/total_batch

    print("Epoch:{:4d},Cost={:.8f}".format(epoch,avg_cost))
        

600
Epoch:   0,Cost=0.50976586
Epoch:   1,Cost=0.31249800
Epoch:   2,Cost=0.27423537
Epoch:   3,Cost=0.24632946
Epoch:   4,Cost=0.22659017
Epoch:   5,Cost=0.24958697
Epoch:   6,Cost=0.21044858
Epoch:   7,Cost=0.19861630
Epoch:   8,Cost=0.21605150
Epoch:   9,Cost=0.19666679
Epoch:  10,Cost=0.24017915


[r:r+1]：

这是一种切片操作，它返回从索引 r 开始到 r+1（不包含 r+1）的子张量。
返回结果的形状仍然保留了“批次维度”（即使只有一个样本），例如原始张量形状为 [N, H, W]，那么 [r:r+1] 得到的形状为 [1, H, W]。

[r]：

直接通过索引取出第 r 个元素，返回结果的形状会丢失批次维度。例如，从 [N, H, W] 中取出第 r 个样本后，其形状直接变为 [H, W]。

In [12]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)
    
    X=mnist_test.data.view(-1,28*28).float().to(device)
    Y=mnist_test.targets.to(device)
    hypothesis=model(X)
    cost=criterion(hypothesis,Y)
    correct=torch.argmax(hypothesis,1)==Y
    accuracy=correct.float().mean()*100
    print(f"Accuracy:{accuracy}%")
    
    r=random.randint(0,len(mnist_test)-1)
    x=mnist_test.data[r:r+1].view(-1,28*28).float().to(device)
    y=mnist_test.targets[r:r+1].to(device)
    hypothesis=model(x)
    print("Prediction:",torch.argmax(hypothesis,1).item())
    print("Label:",y.item())

Accuracy:74.11000061035156%
Prediction: 4
Label: 4
